# MNIST Classification with NN

In [3]:
import torchvision
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn

In [4]:
image_path = '/Users/subashpandey/Desktop/ML-DL-From-Scratch/data/'
transform = transforms.Compose([
    transforms.ToTensor()
])
mnist_train_dataset = torchvision.datasets.MNIST(root=image_path, train=True, transform=transform, download=False)
mnist_test_dataset = torchvision.datasets.MNIST(root=image_path, train=False, transform=transform, download=False)
batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset, batch_size, shuffle=True)

### Constructing the NN model

In [5]:
hidden_units = [32,16]
image_size = mnist_train_dataset[0][0].shape
input_size = image_size[0] * image_size[1] * image_size[2]
all_layers = [nn.Flatten()]
for hidden_unit in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit
all_layers.append(nn.Linear(hidden_units[-1], 10))
model = nn.Sequential(*all_layers)
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=32, bias=True)
  (2): ReLU()
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): ReLU()
  (5): Linear(in_features=16, out_features=10, bias=True)
)

Note that the model starts with a flatten layer that flattens an input image into
a one-dimensional tensor. This is because the input images are in the shape of
[1, 28, 28]. The model has two hidden layers, with 32 and 16 units respectively.
And it ends with an output layer of ten units representing ten classes, activated
by a softmax function.

In [6]:
# training, evaluation, and prediction
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
torch.manual_seed(1)
num_epochs = 20
for epoch in range(num_epochs):
    accuracy_hist_train = 0
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
        accuracy_hist_train += is_correct.sum()
    accuracy_hist_train /= len(train_dl.dataset)
    print(f'Epoch {epoch}  Accuracy '
          f'{accuracy_hist_train:.4f}')

Epoch 0  Accuracy 0.8531
Epoch 1  Accuracy 0.9287
Epoch 2  Accuracy 0.9413
Epoch 3  Accuracy 0.9506
Epoch 4  Accuracy 0.9558
Epoch 5  Accuracy 0.9592
Epoch 6  Accuracy 0.9627
Epoch 7  Accuracy 0.9649
Epoch 8  Accuracy 0.9673
Epoch 9  Accuracy 0.9690
Epoch 10  Accuracy 0.9711
Epoch 11  Accuracy 0.9729
Epoch 12  Accuracy 0.9737
Epoch 13  Accuracy 0.9747
Epoch 14  Accuracy 0.9766
Epoch 15  Accuracy 0.9778
Epoch 16  Accuracy 0.9780
Epoch 17  Accuracy 0.9798
Epoch 18  Accuracy 0.9807
Epoch 19  Accuracy 0.9815


In [7]:
pred = model(mnist_test_dataset.data / 255.)
is_correct = (torch.argmax(pred, dim=1) == mnist_test_dataset.targets).float()
print(f"Test accuracy: {is_correct.mean():.4f}")

Test accuracy: 0.9647


# MNIST Classification using PyTorch Lightning

In [8]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
from torchmetrics import Accuracy

/Users/subashpandey/Library/Caches/pypoetry/virtualenvs/subash-pandey-jvfq-machine-learning-new-vz-Rrc3MWra-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
class MultiLayerPerceptron(pl.LightningModule):
    def __init__(self, image_shape=(1, 28, 28), hidden_units=(32, 16)):
        super().__init__()
        
        # new PL attributes:

        self.train_acc = Accuracy(task="multiclass", num_classes=10)
        self.valid_acc = Accuracy(task="multiclass", num_classes=10)
        self.test_acc = Accuracy(task="multiclass", num_classes=10)

        
        # Model similar to previous section:
        input_size = image_shape[0] * image_shape[1] * image_shape[2] 
        all_layers = [nn.Flatten()]
        for hidden_unit in hidden_units: 
            layer = nn.Linear(input_size, hidden_unit) 
            all_layers.append(layer) 
            all_layers.append(nn.ReLU()) 
            input_size = hidden_unit 
 
        all_layers.append(nn.Linear(hidden_units[-1], 10)) 
        self.model = nn.Sequential(*all_layers)

    def forward(self, x):
        x = self.model(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.train_acc.update(preds, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    # Conditionally define epoch end methods based on PyTorch Lightning version
    def on_training_epoch_end(self):
        self.log("train_acc", self.train_acc.compute())
        self.train_acc.reset()

    def on_validation_epoch_end(self):
        self.log("valid_acc", self.valid_acc.compute())
        self.valid_acc.reset()

    def on_test_epoch_end(self):
        self.log("test_acc", self.test_acc.compute())
        self.test_acc.reset()

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.test_acc.update(preds, y)
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", self.test_acc.compute(), prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [14]:
# LightiningDataModule
from torch.utils.data import DataLoader    
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms

class MnistDataModule(pl.LightningDataModule):
    def __init__(self, data_path='/Users/subashpandey/Desktop/ML-DL-From-Scratch/data/'):
        super().__init__()
        self.data_path = data_path
        self.transform = transforms.Compose([transforms.ToTensor()])

    def prepare_data(self):
        MNIST(root=self.data_path, download=True)

    def setup(self, stage=None):
        # stage is either 'fit', 'validate', 'test', or 'predict'
        mnist_all = MNIST(root=self.data_path, train=True, transform=self.transform, download=False)

        self.train, self.val = random_split(
            mnist_all, [55000,5000], generator=torch.Generator().manual_seed(1)
        )
        self.test = MNIST(root=self.data_path, train=False, transform=self.transform, download=False)

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=64, num_workers=4)
    
    def val_dataloader(self):
        return DataLoader(self.val, batch_size=64, num_workers=4)
    
    def test_dataloader(self):
     return DataLoader(self.test, batch_size=64, num_workers=4)


In [15]:
torch.manual_seed(1)
mnist_dm = MnistDataModule()

In [16]:
# training the model
mnistclassifier = MultiLayerPerceptron()

if torch.cuda.is_available():
    trainer = pl.Trainer(max_epochs=10, gpus=1)
else:
    trainer = pl.Trainer(max_epochs=10)

trainer.fit(model=mnistclassifier, datamodule=mnist_dm)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | train_acc | MulticlassAccuracy | 0      | train
1 | valid_acc | MulticlassAccuracy | 0      | train
2 | test_acc  | MulticlassAccuracy | 0      | train
3 | model     | Sequential         | 25.8 K | train
---------------------------------------------------------
25.8 K    Trainable params
0         Non-trainable params
25.8 K    Total params
0.103     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode
/Users/subashpandey/Library/Caches/pypoetry/virtualenvs/subash-pandey-jvfq-machine-learning-new-vz-Rrc3MWra-py3.12/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initializati

Epoch 9: 100%|██████████| 860/860 [00:07<00:00, 114.78it/s, v_num=2, train_loss=0.248] 

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 860/860 [00:07<00:00, 114.68it/s, v_num=2, train_loss=0.248]


### Evaluating the model using TensorBoard

In [17]:
%load_ext tensorboard
%tensorboard --logdir /Users/subashpandey/Desktop/ML-DL-From-Scratch/miniprojects/lightning_logs/

Reusing TensorBoard on port 6006 (pid 86907), started 0:06:28 ago. (Use '!kill 86907' to kill it.)

In [19]:
from pytorch_lightning.callbacks import ModelCheckpoint


mnistclassifier = MultiLayerPerceptron()

callbacks = [ModelCheckpoint(save_top_k=1, mode='max', monitor="valid_acc")] # save top 1 model

if torch.cuda.is_available(): # if you have GPUs
    trainer = pl.Trainer(max_epochs=10, callbacks=callbacks, gpus=1)
else:
    trainer = pl.Trainer(max_epochs=10, callbacks=callbacks)

trainer.fit(model=mnistclassifier, datamodule=mnist_dm)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/subashpandey/Library/Caches/pypoetry/virtualenvs/subash-pandey-jvfq-machine-learning-new-vz-Rrc3MWra-py3.12/lib/python3.12/site-packages/pytorch_lightning/trainer/configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | train_acc | MulticlassAccuracy | 0      | train
1 | valid_acc | MulticlassAccuracy | 0      | train
2 | test_acc  | MulticlassAccuracy | 0      | train
3 | model     | Sequential         | 25.8 K | train
---------------------------------------------------------
25.8 K    Trainable params
0         Non-trainable params
25.8 K    Total params
0.103     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode
/Users/subashpandey/Library/Cac

Epoch 1:   0%|          | 0/860 [00:00<?, ?it/s, v_num=3, train_loss=0.684]            

/Users/subashpandey/Library/Caches/pypoetry/virtualenvs/subash-pandey-jvfq-machine-learning-new-vz-Rrc3MWra-py3.12/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:384: `ModelCheckpoint(monitor='valid_acc')` could not find the monitored key in the returned metrics: ['train_loss', 'epoch', 'step']. HINT: Did you call `log('valid_acc', value)` in the `LightningModule`?


Epoch 9: 100%|██████████| 860/860 [00:07<00:00, 118.75it/s, v_num=3, train_loss=0.0412] 

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 860/860 [00:07<00:00, 118.73it/s, v_num=3, train_loss=0.0412]


In [ ]:
# evaluating the model using tensorboard
trainer.test(model=mnistclassifier, datamodule=mnist_dm, ckpt_path='best')

In [22]:
#reusing the model
model = MultiLayerPerceptron.load_from_checkpoint('/Users/subashpandey/Desktop/ML-DL-From-Scratch/miniprojects/lightning_logs/version_2/checkpoints/epoch=9-step=8600.ckpt')